In [2]:
# Install library (jalankan jika belum ada)
# !pip install mysql-connector-python beautifulsoup4 requests schedule

import requests
from bs4 import BeautifulSoup
import mysql.connector
import schedule
import time
from datetime import datetime

print("✅ Library Berhasil Diimpor!")

✅ Library Berhasil Diimpor!


In [4]:
db_config = {
    'host': '127.0.0.1',
    'user': 'root',
    'password': '', # Isi jika MySQL kamu ada passwordnya
    'database': 'averroes_scraping'
}

def test_koneksi():
    try:
        conn = mysql.connector.connect(**db_config)
        print("✅ Koneksi ke MySQL Berhasil!")
        conn.close()
    except Exception as e:
        print(f"❌ Koneksi Gagal: {e}\nPastikan MySQL di XAMPP sudah ON.")

test_koneksi()

✅ Koneksi ke MySQL Berhasil!


In [12]:
def jalankan_proses_scraping():
    url = "https://coinvestasi.com/berita" 
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
    }
    
    print(f"\n🌐 [{datetime.now().strftime('%H:%M:%S')}] Scraping Coinvestasi...")
    
    try:
        response = requests.get(url, headers=headers, timeout=15)
        soup = BeautifulSoup(response.text, 'html.parser')
        articles = []
        
        # Sasar elemen artikel berdasarkan struktur visual di gambar kamu
        items = soup.select('article, .post-item, .archive-card, .flex.flex-col.gap-4')

        for item in items:
            title_tag = item.select_one('h1, h2, h3')
            a_tag = title_tag.find('a') if title_tag else item.find('a')
            
            if a_tag and a_tag.get('href') and len(a_tag.text.strip()) > 20:
                judul = a_tag.get_text().strip()
                link = a_tag['href']
                if link.startswith('/'): link = "https://coinvestasi.com" + link
                
                # CARI THUMBNAIL (Targeting wp-content/uploads)
                img_tag = item.find('img')
                thumb = None
                if img_tag:
                    # Cek berbagai atribut tempat WordPress menyimpan URL gambar
                    thumb = (img_tag.get('src') or 
                             img_tag.get('data-src') or 
                             img_tag.get('data-lazy-src') or
                             img_tag.get('srcset', '').split(' ')[0])
                
                # Pastikan thumbnail mengarah ke URL yang valid (bukan file .svg atau kecil)
                if judul and link:
                    articles.append((judul, link, thumb))
        
        if articles:
            unique_data = list(set(articles))
            conn = mysql.connector.connect(**db_config)
            cursor = conn.cursor()
            
            # Hapus data lama agar dashboard admin tidak lemot
            cursor.execute("DELETE FROM berita_crypto WHERE waktu_scraped < NOW() - INTERVAL 1 DAY")
            
            # Masukkan data (REPLACE INTO agar thumbnail terupdate jika berubah)
            query = "REPLACE INTO berita_crypto (judul, url, thumbnail) VALUES (%s, %s, %s)"
            cursor.executemany(query, unique_data[:15])
            
            conn.commit()
            print(f"✨ Berhasil! {cursor.rowcount} berita (termasuk thumbnail) masuk ke MySQL.")
            cursor.close()
            conn.close()
        else:
            print("⚠️ Belum ketemu selektornya. Coinvestasi mungkin pakai proteksi tingkat tinggi.")
            
    except Exception as e:
        print(f"❌ Error: {e}")

# Jalankan manual
jalankan_proses_scraping()


🌐 [09:42:42] Scraping Coinvestasi...
⚠️ Belum ketemu selektornya. Coinvestasi mungkin pakai proteksi tingkat tinggi.
